In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Course 3 : Explosion model taking into account fuel consumption

## One equation case

$$
d_{\tau} \widetilde{\theta} = f(\widetilde{\theta}) = \exp\Bigg(\frac{\widetilde{\theta}}{1 + \widetilde{\theta} \, / \, \beta}\Bigg) \Bigg(1 - \frac{\widetilde{\theta}}{\overline T_r} \Bigg)
$$

In [ ]:
class explosion_with_consumption_1eq_model:

    def __init__(self, Tr, beta):
        self.Tr = Tr
        self.beta = beta 

    def fcn(self, t, theta):
        beta = self.beta
        Tr = self.Tr
        theta_dot = np.exp(theta/(1+(theta/beta))) * (1-theta/Tr) 
        return theta_dot
    
    def jac(self, t, theta):
        beta = self.beta
        Tr = self.Tr
        exp = np.exp(theta/(1+(theta/beta)))
        return (exp/(1+theta/beta)**2) * (1-theta/Tr) - exp/Tr

## Source term

In [ ]:
Tr = 200.0

theta = np.linspace(0, Tr, 1000)

fig = make_subplots(rows=2, cols=1, subplot_titles=("Non-linearity of the source term", "Derivative of source term"))

beta = np.arange(10, 100, 10)
for i, beta_i in enumerate(beta):
    emwc1eq = explosion_with_consumption_1eq_model(Tr, beta_i)
    fcn = emwc1eq.fcn
    jac = emwc1eq.jac
    fig.add_trace(go.Scatter(visible=False, x=theta, y=fcn(0, theta), name='f'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=theta, y=jac(0, theta), name='df/dtheta'), row=2, col=1)
    
fig.data[0].visible = True
fig.data[1].visible = True

# create slider
steps = []
for i, beta_i in enumerate(beta):
    step = dict(method="update", label = f"{beta_i}", 
                args=[{"visible": [(el==2*i) or (el==2*i+1) for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'beta = '}, steps=steps)]

fig.update_layout(sliders=sliders, height=800)
fig.update_xaxes(title_text="theta", row=1, col=1)
fig.update_yaxes(title_text="f(theta)", row=1, col=1)
fig.update_xaxes(title_text="theta", row=2, col=1)
fig.update_yaxes(title_text="df/dtheta", row=2, col=1)
fig['layout']['sliders'][0]['pad']=dict(t= 50)
fig.show()

# Solution

In [ ]:
class ode_result:
    def __init__(self, y, t):
        self.y = y
        self.t = t

def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*np.array(fcn(tn, yn))

    return ode_result(y, t)

def backward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    def g(uip1, *args):
        uip, tip1 = args
        return uip1 - uip - dt*np.array(fcn(tip1, uip1))

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y0 = yn + dt*np.array(fcn(tn, yn))
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0
        sol = root(g, y0, (yn, tn+dt))
        y[:,it+1] = sol.x

    return ode_result(y, t)

In [ ]:
Tr = 200.0
beta = 10. # 10 très illustratif avec nt = 501 / 401 / 301 / 201 / 171

emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
fcn = emwc1eq.fcn
jac = emwc1eq.jac

tini = 0.0
tend = 5.0
nt = np.array([171, 201, 401, 501])

yini = (0.,)

fig =  make_subplots(rows=3, cols=1, subplot_titles=("Solution", "Eigen value", "Eigen value . dt"))


for i, nt_i in enumerate(nt):
    
    dt = (tend-tini) / (nt_i-1)

    emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
    fcn = emwc1eq.fcn
    jac = emwc1eq.jac

    sol_ref = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)
    sol_fe = forward_euler(tini, tend, nt_i, yini, fcn)
    sol_be = backward_euler(tini, tend, nt_i, yini, fcn)

    fig.add_trace(go.Scatter(visible=False, x=sol_ref.t, y=sol_ref.y[0],  line_color="green", legendgroup='sol', 
                         legendgrouptitle_text="Solution", name='ref sol'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=sol_fe.y[0], line_color="blue", legendgroup='sol', 
                         legendgrouptitle_text="Solution", name='forward euler'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_be.t, y=sol_be.y[0], line_color="red", legendgroup='sol', 
                         legendgrouptitle_text="Solution", name='backward_euler'), row=1, col=1)

    eig_vals_ref = np.zeros(sol_ref.t.size)
    for it in range(sol_ref.t.size):
        eig_vals_ref[it] = jac(0, sol_ref.y[:,it])
    eig_vals_fe = np.zeros(sol_fe.t.size)
    for it in range(sol_be.t.size):
        eig_vals_fe[it] = jac(0, sol_fe.y[:,it])
    eig_vals_be = np.zeros(sol_be.t.size)
    for it in range(sol_be.t.size):
        eig_vals_be[it] = jac(0, sol_be.y[:,it])

    fig.add_trace(go.Scatter(visible=False, x=sol_ref.t, y=eig_vals_ref, line_color="green", legendgroup='eig', 
                         legendgrouptitle_text="Eigen value", name='ref sol'), row=2, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=eig_vals_fe, line_color="blue", legendgroup='eig', 
                         legendgrouptitle_text="Eigen value", name='forward euler'), row=2, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_be.t, y=eig_vals_be, line_color="red", legendgroup='eig', 
                         legendgrouptitle_text="Eigen value", name='backward_euler'), row=2, col=1)

    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=eig_vals_fe*dt, line_color="blue", legendgroup='eigdt', 
                         legendgrouptitle_text="Eigen value . dt", name='forward euler'), row=3, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_be.t, y=eig_vals_be*dt, line_color="red", legendgroup='eigdt', 
                         legendgrouptitle_text="Eigen value . dt",  name='backward_euler'), row=3, col=1)

for i in range(3*8, 4*8): fig.data[i].visible = True

# create slider
steps = []
for i, nt_i in enumerate(nt):
    step = dict(method="update", label = f"{nt_i}", 
                args=[{"visible": [(el==8*i) or (el==8*i+1) or (el==8*i+2) or (el==8*i+3) or (el==8*i+4) or (el==8*i+5)
                                   or (el==8*i+6) or (el==8*i+7)  for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(active=3, currentvalue={'prefix': 'nt = '}, steps=steps)]

fig.update_layout(sliders=sliders, height=1000)
fig.show()